In [75]:
!pip install mne

In [2]:
import mne
import numpy as np
import pandas as pd

## Data Loading and Inspection 

In [3]:
path = '../Datasets/EEG_data_BRMH.csv'
input= pd.read_csv(path)
input.info()
input

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Columns: 1149 entries, no. to COH.F.gamma.r.O1.s.O2
dtypes: float64(1144), int64(1), object(4)
memory usage: 8.3+ MB


,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,941,M,22.0,2014.8.28,13.0,116.0,Healthy control,Healthy control,41.851823,36.771496,...,82.905657,34.850706,63.970519,63.982003,51.244725,62.203684,62.062237,31.013031,31.183413,98.325230
941,942,M,26.0,2014.9.19,13.0,118.0,Healthy control,Healthy control,18.986856,19.401387,...,65.917918,66.700117,44.756285,49.787513,98.905995,54.021304,93.902401,52.740396,92.807331,56.320868
942,943,M,26.0,2014.9.27,16.0,113.0,Healthy control,Healthy control,28.781317,32.369230,...,61.040959,27.632209,45.552852,33.638817,46.690983,19.382928,41.050717,7.045821,41.962451,19.092111
943,944,M,24.0,2014.9.20,13.0,107.0,Healthy control,Healthy control,19.929100,25.196375,...,99.113664,48.328934,41.248470,28.192238,48.665743,42.007147,28.735945,27.176500,27.529522,20.028446


In [7]:
input[input['specific.disorder'] == 'Schizophrenia']

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
283,284,M,29.22,2012.1.11,16.0,102.0,Schizophrenia,Schizophrenia,13.533164,14.711985,...,66.019853,43.970020,46.364561,51.988403,58.615577,36.998060,55.852831,30.335654,54.374133,47.250159
290,291,M,18.77,2013.4.22,12.0,95.0,Schizophrenia,Schizophrenia,7.970252,7.266967,...,64.171960,70.035610,60.114549,56.457333,62.986150,44.837748,51.857004,53.691281,62.306837,48.211705
291,292,M,24.53,2011.1.17,15.0,114.0,Schizophrenia,Schizophrenia,8.269064,10.083543,...,62.572653,41.940265,34.265888,51.330723,63.553482,24.377419,59.216170,11.659469,62.701330,28.049332
292,293,F,23.28,2011.1.10,16.0,94.0,Schizophrenia,Schizophrenia,24.920269,23.793912,...,75.048447,36.701346,40.122985,51.511086,40.950687,45.449571,63.396844,17.737763,29.802046,49.315534
298,299,M,56.51,2011.2.14,12.0,97.0,Schizophrenia,Schizophrenia,22.115575,19.107165,...,70.981709,38.617658,31.883457,37.845721,53.874866,33.885331,47.987555,24.885383,50.268147,48.232344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,905,F,24.13,2016.1.18,10.0,66.0,Schizophrenia,Schizophrenia,8.850259,7.932767,...,77.871506,67.417227,75.515818,56.525824,71.655885,40.089206,46.900782,39.277051,80.062799,38.444530
908,909,M,20.06,2016.12.5,14.0,80.0,Schizophrenia,Schizophrenia,44.905342,55.740004,...,90.378518,89.826264,70.512948,81.017979,99.503053,56.282745,91.010983,55.875218,91.168993,64.238404
911,912,F,31.34,2014.5.13,12.0,60.0,Schizophrenia,Schizophrenia,23.812315,21.048566,...,61.293685,22.751652,60.428247,40.464199,61.489809,32.362762,70.449612,16.123001,77.032637,38.749749
914,915,M,28.72,2015.6.4,16.0,99.0,Schizophrenia,Schizophrenia,16.087604,15.842329,...,86.800565,66.240201,70.120359,68.885433,87.524891,58.633800,78.457544,51.906669,80.270965,69.926903


In [78]:
input.describe()

,no.,age,education,IQ,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,AB.A.delta.f.F4,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
count,945.000000,945.000000,930.000000,932.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,...,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000
mean,473.000000,30.594804,13.438710,101.580472,20.182936,21.177584,17.749553,18.901698,20.447079,19.490329,...,75.889633,55.049113,56.959796,60.739169,69.829254,47.862489,66.832798,39.301406,66.153213,57.056207
std,272.942302,11.781592,2.550461,17.022414,11.282022,12.230662,10.003598,9.079482,9.742912,9.252500,...,15.861641,19.541956,18.326785,18.052887,17.725321,19.685722,17.028701,20.790933,18.088548,19.600107
min,1.000000,18.000000,0.000000,49.000000,3.272260,3.244199,3.050507,3.412618,5.066593,4.048931,...,12.611954,0.519048,6.926792,0.708008,2.421748,0.036664,1.032207,1.228502,0.363268,3.988805
25%,237.000000,21.730000,12.000000,91.000000,12.784872,13.019269,11.134327,12.460586,13.548645,12.637717,...,68.014443,41.607506,43.682444,48.374883,58.937785,32.581046,55.872070,22.049743,54.710605,43.955229
50%,473.000000,26.150000,13.000000,102.000000,17.065286,17.838251,15.541469,16.733004,18.065276,17.432568,...,78.647712,55.158858,56.657348,61.257972,72.298636,45.719426,68.238375,36.549938,67.988937,57.515871
75%,709.000000,35.450000,16.000000,114.000000,24.492760,25.654394,21.623961,23.347900,25.573096,23.956748,...,87.398387,69.740640,70.649245,73.979100,83.066877,62.081839,79.192418,54.169209,79.527764,71.626382
max,945.000000,71.880000,20.000000,145.000000,92.826192,101.515687,91.373456,82.544167,101.566662,77.197502,...,99.678649,100.000000,99.307895,100.000000,99.581629,98.720067,99.650154,98.413320,100.000000,99.287092


In [79]:
print(input.groupby('specific.disorder').count()['no.'])

specific.disorder
Acute stress disorder              38
Adjustment disorder                38
Alcohol use disorder               93
Behavioral addiction disorder      93
Bipolar disorder                   67
Depressive disorder               199
Healthy control                    95
Obsessive compulsitve disorder     46
Panic disorder                     59
Posttraumatic stress disorder      52
Schizophrenia                     117
Social anxiety disorder            48
Name: no., dtype: int64


## Cleaning and Preprocessing

In [80]:
ptsd = input[input['specific.disorder'] == 'Posttraumatic stress disorder']
hc = input[input['specific.disorder'] == 'Healthy control'].sample(n=52, random_state=42)
data = pd.concat([ptsd, hc])
data

,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
297,298,F,36.62,2011.1.28,12.0,99.0,Trauma and stress related disorder,Posttraumatic stress disorder,17.603385,17.243334,...,94.727123,73.994841,75.159075,72.479599,81.756815,74.578657,78.980627,60.508215,65.113378,69.238767
309,310,F,55.21,2012.4.9,16.0,120.0,Trauma and stress related disorder,Posttraumatic stress disorder,21.714048,19.579805,...,83.821476,61.617403,63.964423,73.096645,74.161249,50.432291,77.440228,35.794362,70.592405,48.283439
310,311,F,26.93,2011.3.24,16.0,116.0,Trauma and stress related disorder,Posttraumatic stress disorder,13.371076,14.028142,...,70.585726,63.864062,46.127953,65.675414,62.455949,33.114945,56.163998,37.477109,72.732968,46.665464
315,316,M,26.71,2012.7.3,17.0,137.0,Trauma and stress related disorder,Posttraumatic stress disorder,30.473244,13.954586,...,87.768539,71.799769,65.845016,67.734909,78.328065,58.141474,68.113967,53.391012,71.111448,67.598506
318,319,F,23.45,2014.6.2,12.0,89.0,Trauma and stress related disorder,Posttraumatic stress disorder,18.488575,19.603144,...,70.188502,27.698816,24.181746,34.445899,47.797466,20.737129,47.210870,11.104500,50.442443,36.952529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,138,F,29.04,2018.8.9,16.0,114.0,Healthy control,Healthy control,22.737005,23.110192,...,88.188438,48.337650,77.414168,76.804444,51.999675,76.470883,80.365068,48.841509,56.730760,82.017645
116,117,F,25.65,2018.7.2,18.0,118.0,Healthy control,Healthy control,20.081892,17.928614,...,99.447826,99.424714,44.181841,61.683927,99.513428,43.007308,62.525177,42.766646,62.843978,39.895496
130,131,F,29.86,2018.7.19,20.0,113.0,Healthy control,Healthy control,46.306229,47.379694,...,90.988675,82.588144,91.869140,88.146513,82.027254,87.799612,85.360432,84.867957,90.909785,90.730560
184,185,M,27.00,2017.4.29,18.0,130.0,Healthy control,Healthy control,22.152399,22.698280,...,58.751205,46.407702,55.207337,50.262542,56.166800,37.817569,55.401371,31.634053,67.817508,43.937626


In [81]:
separator = data.columns.get_loc(data.columns[data.isnull().all()][0])
separator

122

In [82]:
data.drop('no.', axis=1, inplace=True)
data.drop('eeg.date', axis=1, inplace=True)
data.drop('main.disorder', axis=1, inplace=True)
data.drop('education', axis=1, inplace=True)
data.reset_index(drop=True, inplace=True)
data.drop('index', axis=1, errors='ignore')
data.index = range(1, len(data) + 1)
data

,sex,age,IQ,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,AB.A.delta.f.F4,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
1,F,36.62,99.0,Posttraumatic stress disorder,17.603385,17.243334,11.729942,15.381709,18.307108,28.369365,...,94.727123,73.994841,75.159075,72.479599,81.756815,74.578657,78.980627,60.508215,65.113378,69.238767
2,F,55.21,120.0,Posttraumatic stress disorder,21.714048,19.579805,18.522271,26.820075,25.429025,27.345290,...,83.821476,61.617403,63.964423,73.096645,74.161249,50.432291,77.440228,35.794362,70.592405,48.283439
3,F,26.93,116.0,Posttraumatic stress disorder,13.371076,14.028142,16.901291,21.767857,18.227411,16.825877,...,70.585726,63.864062,46.127953,65.675414,62.455949,33.114945,56.163998,37.477109,72.732968,46.665464
4,M,26.71,137.0,Posttraumatic stress disorder,30.473244,13.954586,23.056715,26.177810,27.884679,24.520958,...,87.768539,71.799769,65.845016,67.734909,78.328065,58.141474,68.113967,53.391012,71.111448,67.598506
5,F,23.45,89.0,Posttraumatic stress disorder,18.488575,19.603144,9.899157,21.724115,32.700666,24.510528,...,70.188502,27.698816,24.181746,34.445899,47.797466,20.737129,47.210870,11.104500,50.442443,36.952529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,F,29.04,114.0,Healthy control,22.737005,23.110192,20.655895,18.224802,19.315381,16.482197,...,88.188438,48.337650,77.414168,76.804444,51.999675,76.470883,80.365068,48.841509,56.730760,82.017645
101,F,25.65,118.0,Healthy control,20.081892,17.928614,15.977752,18.119963,17.029861,13.545934,...,99.447826,99.424714,44.181841,61.683927,99.513428,43.007308,62.525177,42.766646,62.843978,39.895496
102,F,29.86,113.0,Healthy control,46.306229,47.379694,48.808082,50.943346,43.768029,41.659826,...,90.988675,82.588144,91.869140,88.146513,82.027254,87.799612,85.360432,84.867957,90.909785,90.730560
103,M,27.00,130.0,Healthy control,22.152399,22.698280,19.281922,28.462510,31.292110,27.909515,...,58.751205,46.407702,55.207337,50.262542,56.166800,37.817569,55.401371,31.634053,67.817508,43.937626


In [83]:
data

,sex,age,IQ,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,AB.A.delta.d.F3,AB.A.delta.e.Fz,AB.A.delta.f.F4,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
1,F,36.62,99.0,Posttraumatic stress disorder,17.603385,17.243334,11.729942,15.381709,18.307108,28.369365,...,94.727123,73.994841,75.159075,72.479599,81.756815,74.578657,78.980627,60.508215,65.113378,69.238767
2,F,55.21,120.0,Posttraumatic stress disorder,21.714048,19.579805,18.522271,26.820075,25.429025,27.345290,...,83.821476,61.617403,63.964423,73.096645,74.161249,50.432291,77.440228,35.794362,70.592405,48.283439
3,F,26.93,116.0,Posttraumatic stress disorder,13.371076,14.028142,16.901291,21.767857,18.227411,16.825877,...,70.585726,63.864062,46.127953,65.675414,62.455949,33.114945,56.163998,37.477109,72.732968,46.665464
4,M,26.71,137.0,Posttraumatic stress disorder,30.473244,13.954586,23.056715,26.177810,27.884679,24.520958,...,87.768539,71.799769,65.845016,67.734909,78.328065,58.141474,68.113967,53.391012,71.111448,67.598506
5,F,23.45,89.0,Posttraumatic stress disorder,18.488575,19.603144,9.899157,21.724115,32.700666,24.510528,...,70.188502,27.698816,24.181746,34.445899,47.797466,20.737129,47.210870,11.104500,50.442443,36.952529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,F,29.04,114.0,Healthy control,22.737005,23.110192,20.655895,18.224802,19.315381,16.482197,...,88.188438,48.337650,77.414168,76.804444,51.999675,76.470883,80.365068,48.841509,56.730760,82.017645
101,F,25.65,118.0,Healthy control,20.081892,17.928614,15.977752,18.119963,17.029861,13.545934,...,99.447826,99.424714,44.181841,61.683927,99.513428,43.007308,62.525177,42.766646,62.843978,39.895496
102,F,29.86,113.0,Healthy control,46.306229,47.379694,48.808082,50.943346,43.768029,41.659826,...,90.988675,82.588144,91.869140,88.146513,82.027254,87.799612,85.360432,84.867957,90.909785,90.730560
103,M,27.00,130.0,Healthy control,22.152399,22.698280,19.281922,28.462510,31.292110,27.909515,...,58.751205,46.407702,55.207337,50.262542,56.166800,37.817569,55.401371,31.634053,67.817508,43.937626


In [84]:
data.columns = data.columns.str.strip().str.lower()
print(data.columns)

Index(['sex', 'age', 'iq', 'specific.disorder', 'ab.a.delta.a.fp1',
       'ab.a.delta.b.fp2', 'ab.a.delta.c.f7', 'ab.a.delta.d.f3',
       'ab.a.delta.e.fz', 'ab.a.delta.f.f4',
       ...
       'coh.f.gamma.o.pz.p.p4', 'coh.f.gamma.o.pz.q.t6',
       'coh.f.gamma.o.pz.r.o1', 'coh.f.gamma.o.pz.s.o2',
       'coh.f.gamma.p.p4.q.t6', 'coh.f.gamma.p.p4.r.o1',
       'coh.f.gamma.p.p4.s.o2', 'coh.f.gamma.q.t6.r.o1',
       'coh.f.gamma.q.t6.s.o2', 'coh.f.gamma.r.o1.s.o2'],
      dtype='object', length=1145)


In [85]:
data['iq'] = data['iq'].astype('float64')
data['specific.disorder'] = data['specific.disorder'].astype('category')
data['sex'] = data['sex'].astype('category')

print(data.dtypes.head())  

sex                  category
age                   float64
iq                    float64
specific.disorder    category
ab.a.delta.a.fp1      float64
dtype: object


In [86]:
AB_data = data.iloc[:,:separator]
AB_data

,sex,age,iq,specific.disorder,ab.a.delta.a.fp1,ab.a.delta.b.fp2,ab.a.delta.c.f7,ab.a.delta.d.f3,ab.a.delta.e.fz,ab.a.delta.f.f4,...,ab.f.gamma.n.p3,ab.f.gamma.o.pz,ab.f.gamma.p.p4,ab.f.gamma.q.t6,ab.f.gamma.r.o1,ab.f.gamma.s.o2,unnamed: 122,coh.a.delta.a.fp1.b.fp2,coh.a.delta.a.fp1.c.f7,coh.a.delta.a.fp1.d.f3
1,F,36.62,99.0,Posttraumatic stress disorder,17.603385,17.243334,11.729942,15.381709,18.307108,28.369365,...,0.909426,0.721567,0.794328,0.728853,0.745882,0.818206,NaN,80.375130,22.109628,83.707393
2,F,55.21,120.0,Posttraumatic stress disorder,21.714048,19.579805,18.522271,26.820075,25.429025,27.345290,...,0.394191,0.435409,0.410746,0.281834,0.334405,0.318448,NaN,99.699970,60.286374,78.790309
3,F,26.93,116.0,Posttraumatic stress disorder,13.371076,14.028142,16.901291,21.767857,18.227411,16.825877,...,0.571112,0.673131,0.885947,0.381586,0.417947,0.333066,NaN,88.131324,77.536120,82.648529
4,M,26.71,137.0,Posttraumatic stress disorder,30.473244,13.954586,23.056715,26.177810,27.884679,24.520958,...,2.323228,0.853186,0.936312,0.895048,10.464573,1.541601,NaN,69.998866,64.997577,78.672407
5,F,23.45,89.0,Posttraumatic stress disorder,18.488575,19.603144,9.899157,21.724115,32.700666,24.510528,...,1.369314,1.441666,1.420502,0.580418,2.445275,0.880921,NaN,60.305588,58.643782,65.075301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,F,29.04,114.0,Healthy control,22.737005,23.110192,20.655895,18.224802,19.315381,16.482197,...,1.722680,1.863039,2.212353,1.571332,1.974860,7.184206,NaN,54.615410,51.608236,61.993370
101,F,25.65,118.0,Healthy control,20.081892,17.928614,15.977752,18.119963,17.029861,13.545934,...,0.800209,0.911272,0.823876,0.676607,0.654119,0.682812,NaN,90.006558,99.877118,99.868472
102,F,29.86,113.0,Healthy control,46.306229,47.379694,48.808082,50.943346,43.768029,41.659826,...,3.350566,3.561434,3.087733,2.223389,2.907973,2.761723,NaN,26.770053,23.884052,36.898819
103,M,27.00,130.0,Healthy control,22.152399,22.698280,19.281922,28.462510,31.292110,27.909515,...,1.487806,1.530798,1.531705,1.553039,1.733781,2.511217,NaN,51.903003,99.701376,99.778920


In [87]:
COH_data = data.iloc[:, list(range(4)) + list(range(separator+1 , data.shape[1]))]
COH_data

,sex,age,iq,specific.disorder,coh.a.delta.a.fp1.f.f4,coh.a.delta.a.fp1.g.f8,coh.a.delta.a.fp1.h.t3,coh.a.delta.a.fp1.i.c3,coh.a.delta.a.fp1.j.cz,coh.a.delta.a.fp1.k.c4,...,coh.f.gamma.o.pz.p.p4,coh.f.gamma.o.pz.q.t6,coh.f.gamma.o.pz.r.o1,coh.f.gamma.o.pz.s.o2,coh.f.gamma.p.p4.q.t6,coh.f.gamma.p.p4.r.o1,coh.f.gamma.p.p4.s.o2,coh.f.gamma.q.t6.r.o1,coh.f.gamma.q.t6.s.o2,coh.f.gamma.r.o1.s.o2
1,F,36.62,99.0,Posttraumatic stress disorder,65.231645,39.669620,9.881328,41.451435,80.070297,51.082581,...,94.727123,73.994841,75.159075,72.479599,81.756815,74.578657,78.980627,60.508215,65.113378,69.238767
2,F,55.21,120.0,Posttraumatic stress disorder,66.957990,32.227674,35.323545,51.153409,46.000557,45.788812,...,83.821476,61.617403,63.964423,73.096645,74.161249,50.432291,77.440228,35.794362,70.592405,48.283439
3,F,26.93,116.0,Posttraumatic stress disorder,72.210662,59.191032,40.605777,40.376361,48.283716,48.164265,...,70.585726,63.864062,46.127953,65.675414,62.455949,33.114945,56.163998,37.477109,72.732968,46.665464
4,M,26.71,137.0,Posttraumatic stress disorder,54.415283,27.894286,46.780735,51.735236,41.647626,38.914928,...,87.768539,71.799769,65.845016,67.734909,78.328065,58.141474,68.113967,53.391012,71.111448,67.598506
5,F,23.45,89.0,Posttraumatic stress disorder,49.836470,25.962092,31.673369,40.891145,28.835594,42.857171,...,70.188502,27.698816,24.181746,34.445899,47.797466,20.737129,47.210870,11.104500,50.442443,36.952529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,F,29.04,114.0,Healthy control,38.864399,4.053128,58.184960,29.265874,42.911752,31.873634,...,88.188438,48.337650,77.414168,76.804444,51.999675,76.470883,80.365068,48.841509,56.730760,82.017645
101,F,25.65,118.0,Healthy control,69.565596,69.483540,46.104637,56.184711,97.235879,50.862490,...,99.447826,99.424714,44.181841,61.683927,99.513428,43.007308,62.525177,42.766646,62.843978,39.895496
102,F,29.86,113.0,Healthy control,37.182856,4.849569,15.257130,30.057984,24.538788,27.652505,...,90.988675,82.588144,91.869140,88.146513,82.027254,87.799612,85.360432,84.867957,90.909785,90.730560
103,M,27.00,130.0,Healthy control,78.079135,64.240750,82.634276,64.329245,61.943230,61.960092,...,58.751205,46.407702,55.207337,50.262542,56.166800,37.817569,55.401371,31.634053,67.817508,43.937626


In [88]:
data

,sex,age,iq,specific.disorder,ab.a.delta.a.fp1,ab.a.delta.b.fp2,ab.a.delta.c.f7,ab.a.delta.d.f3,ab.a.delta.e.fz,ab.a.delta.f.f4,...,coh.f.gamma.o.pz.p.p4,coh.f.gamma.o.pz.q.t6,coh.f.gamma.o.pz.r.o1,coh.f.gamma.o.pz.s.o2,coh.f.gamma.p.p4.q.t6,coh.f.gamma.p.p4.r.o1,coh.f.gamma.p.p4.s.o2,coh.f.gamma.q.t6.r.o1,coh.f.gamma.q.t6.s.o2,coh.f.gamma.r.o1.s.o2
1,F,36.62,99.0,Posttraumatic stress disorder,17.603385,17.243334,11.729942,15.381709,18.307108,28.369365,...,94.727123,73.994841,75.159075,72.479599,81.756815,74.578657,78.980627,60.508215,65.113378,69.238767
2,F,55.21,120.0,Posttraumatic stress disorder,21.714048,19.579805,18.522271,26.820075,25.429025,27.345290,...,83.821476,61.617403,63.964423,73.096645,74.161249,50.432291,77.440228,35.794362,70.592405,48.283439
3,F,26.93,116.0,Posttraumatic stress disorder,13.371076,14.028142,16.901291,21.767857,18.227411,16.825877,...,70.585726,63.864062,46.127953,65.675414,62.455949,33.114945,56.163998,37.477109,72.732968,46.665464
4,M,26.71,137.0,Posttraumatic stress disorder,30.473244,13.954586,23.056715,26.177810,27.884679,24.520958,...,87.768539,71.799769,65.845016,67.734909,78.328065,58.141474,68.113967,53.391012,71.111448,67.598506
5,F,23.45,89.0,Posttraumatic stress disorder,18.488575,19.603144,9.899157,21.724115,32.700666,24.510528,...,70.188502,27.698816,24.181746,34.445899,47.797466,20.737129,47.210870,11.104500,50.442443,36.952529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,F,29.04,114.0,Healthy control,22.737005,23.110192,20.655895,18.224802,19.315381,16.482197,...,88.188438,48.337650,77.414168,76.804444,51.999675,76.470883,80.365068,48.841509,56.730760,82.017645
101,F,25.65,118.0,Healthy control,20.081892,17.928614,15.977752,18.119963,17.029861,13.545934,...,99.447826,99.424714,44.181841,61.683927,99.513428,43.007308,62.525177,42.766646,62.843978,39.895496
102,F,29.86,113.0,Healthy control,46.306229,47.379694,48.808082,50.943346,43.768029,41.659826,...,90.988675,82.588144,91.869140,88.146513,82.027254,87.799612,85.360432,84.867957,90.909785,90.730560
103,M,27.00,130.0,Healthy control,22.152399,22.698280,19.281922,28.462510,31.292110,27.909515,...,58.751205,46.407702,55.207337,50.262542,56.166800,37.817569,55.401371,31.634053,67.817508,43.937626


In [89]:
data.isnull().sum().sum()

104

In [90]:
test=data.iloc[:,:4]
test

,sex,age,iq,specific.disorder
1,F,36.62,99.0,Posttraumatic stress disorder
2,F,55.21,120.0,Posttraumatic stress disorder
3,F,26.93,116.0,Posttraumatic stress disorder
4,M,26.71,137.0,Posttraumatic stress disorder
5,F,23.45,89.0,Posttraumatic stress disorder
...,...,...,...,...
100,F,29.04,114.0,Healthy control
101,F,25.65,118.0,Healthy control
102,F,29.86,113.0,Healthy control
103,M,27.00,130.0,Healthy control


## Classification

In [91]:
def classify(model, X_train, y_train, X_test, y_test):
    model.fit(X_train,y_train)
    return evaluate_model(model, X_test, y_test)

In [92]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, f1_score, precision_score, classification_report

def evaluate_model(model, x, y_true):
    y_pred = model.predict(x)
    
    print(model)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy:.2f}")

    # Display the classification report
    report = classification_report(y_test, y_pred)
    print("Classification Report:\n", report)
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1 Score': f1_score(y_true, y_pred),
        'Confusion Matrix': confusion_matrix(y_true, y_pred)}

In [93]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer

label_encoder = LabelEncoder()
imputer = SimpleImputer(strategy='mean')

eeg_data = pd.read_csv('../Datasets/dataph.csv')
eeg_data_cleaned = eeg_data.drop(columns=['Unnamed: 122'])
eeg_data_cleaned = eeg_data_cleaned.drop(columns=['eeg.date'])
eeg_data_cleaned['sex'] = label_encoder.fit_transform(eeg_data['sex'].astype(str))
eeg_data_cleaned['specific.disorder'] = label_encoder.fit_transform(eeg_data['specific.disorder'].astype(str))

# Handle any remaining missing values using SimpleImputer
eeg_data_imputed = pd.DataFrame(imputer.fit_transform(eeg_data_cleaned), columns=eeg_data_cleaned.columns)
eeg_data_imputed['sex'] = label_encoder.fit_transform(eeg_data_imputed['sex'].astype(str))
eeg_data_imputed['specific.disorder'] = label_encoder.fit_transform(eeg_data_imputed['specific.disorder'].astype(str))

X = eeg_data_imputed.drop(columns=['specific.disorder'])
y = eeg_data_imputed['specific.disorder']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Support Vector Classifier': SVC(probability=True, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

results ={}
for name in models:
    results[f'{name}'] = classify(models[name], X_train, y_train, X_test, y_test)


results

RandomForestClassifier(random_state=42)
Model Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       0.85      1.00      0.92        11

    accuracy                           0.90        21
   macro avg       0.92      0.90      0.90        21
weighted avg       0.92      0.90      0.90        21

GradientBoostingClassifier(random_state=42)
Model Accuracy: 1.00
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        11

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21

SVC(probability=True, random_state=42)
Model Accuracy: 0.76
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.70

{'Random Forest': {'Accuracy': 0.9047619047619048,
  'Precision': 0.8461538461538461,
  'Recall': 1.0,
  'F1 Score': 0.9166666666666666,
  'Confusion Matrix': array([[ 8,  2],
         [ 0, 11]])},
 'Gradient Boosting': {'Accuracy': 1.0,
  'Precision': 1.0,
  'Recall': 1.0,
  'F1 Score': 1.0,
  'Confusion Matrix': array([[10,  0],
         [ 0, 11]])},
 'Support Vector Classifier': {'Accuracy': 0.7619047619047619,
  'Precision': 0.75,
  'Recall': 0.8181818181818182,
  'F1 Score': 0.7826086956521738,
  'Confusion Matrix': array([[7, 3],
         [2, 9]])},
 'K-Nearest Neighbors': {'Accuracy': 0.7619047619047619,
  'Precision': 0.6875,
  'Recall': 1.0,
  'F1 Score': 0.8148148148148148,
  'Confusion Matrix': array([[ 5,  5],
         [ 0, 11]])}}

In [94]:
eeg_data = data.iloc[:,3:]
eeg_data

,specific.disorder,ab.a.delta.a.fp1,ab.a.delta.b.fp2,ab.a.delta.c.f7,ab.a.delta.d.f3,ab.a.delta.e.fz,ab.a.delta.f.f4,ab.a.delta.g.f8,ab.a.delta.h.t3,ab.a.delta.i.c3,...,coh.f.gamma.o.pz.p.p4,coh.f.gamma.o.pz.q.t6,coh.f.gamma.o.pz.r.o1,coh.f.gamma.o.pz.s.o2,coh.f.gamma.p.p4.q.t6,coh.f.gamma.p.p4.r.o1,coh.f.gamma.p.p4.s.o2,coh.f.gamma.q.t6.r.o1,coh.f.gamma.q.t6.s.o2,coh.f.gamma.r.o1.s.o2
1,Posttraumatic stress disorder,17.603385,17.243334,11.729942,15.381709,18.307108,28.369365,11.764645,7.650070,12.320080,...,94.727123,73.994841,75.159075,72.479599,81.756815,74.578657,78.980627,60.508215,65.113378,69.238767
2,Posttraumatic stress disorder,21.714048,19.579805,18.522271,26.820075,25.429025,27.345290,16.839109,16.482528,21.870727,...,83.821476,61.617403,63.964423,73.096645,74.161249,50.432291,77.440228,35.794362,70.592405,48.283439
3,Posttraumatic stress disorder,13.371076,14.028142,16.901291,21.767857,18.227411,16.825877,17.859026,20.892738,12.853055,...,70.585726,63.864062,46.127953,65.675414,62.455949,33.114945,56.163998,37.477109,72.732968,46.665464
4,Posttraumatic stress disorder,30.473244,13.954586,23.056715,26.177810,27.884679,24.520958,19.931770,16.350662,20.175206,...,87.768539,71.799769,65.845016,67.734909,78.328065,58.141474,68.113967,53.391012,71.111448,67.598506
5,Posttraumatic stress disorder,18.488575,19.603144,9.899157,21.724115,32.700666,24.510528,13.788542,7.538543,14.280676,...,70.188502,27.698816,24.181746,34.445899,47.797466,20.737129,47.210870,11.104500,50.442443,36.952529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Healthy control,22.737005,23.110192,20.655895,18.224802,19.315381,16.482197,15.050377,10.358503,12.321945,...,88.188438,48.337650,77.414168,76.804444,51.999675,76.470883,80.365068,48.841509,56.730760,82.017645
101,Healthy control,20.081892,17.928614,15.977752,18.119963,17.029861,13.545934,11.612536,13.578299,13.790407,...,99.447826,99.424714,44.181841,61.683927,99.513428,43.007308,62.525177,42.766646,62.843978,39.895496
102,Healthy control,46.306229,47.379694,48.808082,50.943346,43.768029,41.659826,28.124478,29.444817,50.549412,...,90.988675,82.588144,91.869140,88.146513,82.027254,87.799612,85.360432,84.867957,90.909785,90.730560
103,Healthy control,22.152399,22.698280,19.281922,28.462510,31.292110,27.909515,19.878541,12.891310,24.809049,...,58.751205,46.407702,55.207337,50.262542,56.166800,37.817569,55.401371,31.634053,67.817508,43.937626


In [95]:
eeg_data.drop(columns='unnamed: 122', inplace=True)
eeg_data.isnull().sum().sum()

0

In [96]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer


label_encoder = LabelEncoder()
imputer = SimpleImputer(strategy='mean')

eeg_data_cleaned['specific.disorder'] = label_encoder.fit_transform(eeg_data['specific.disorder'].astype(str))

# Handle any remaining missing values using SimpleImputer
eeg_data_imputed = pd.DataFrame(imputer.fit_transform(eeg_data_cleaned), columns=eeg_data_cleaned.columns)
eeg_data_imputed['specific.disorder'] = label_encoder.fit_transform(eeg_data_imputed['specific.disorder'].astype(str))

X = eeg_data_imputed.drop(columns=['specific.disorder'])
y = eeg_data_imputed['specific.disorder']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Support Vector Classifier': SVC(probability=True, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

results ={}
for name in models:
    results[f'{name}'] = classify(models[name], X_train, y_train, X_test, y_test) 
results

RandomForestClassifier(random_state=42)
Model Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       0.85      1.00      0.92        11

    accuracy                           0.90        21
   macro avg       0.92      0.90      0.90        21
weighted avg       0.92      0.90      0.90        21

GradientBoostingClassifier(random_state=42)
Model Accuracy: 1.00
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        11

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21

SVC(probability=True, random_state=42)
Model Accuracy: 0.76
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.70

{'Random Forest': {'Accuracy': 0.9047619047619048,
  'Precision': 0.8461538461538461,
  'Recall': 1.0,
  'F1 Score': 0.9166666666666666,
  'Confusion Matrix': array([[ 8,  2],
         [ 0, 11]])},
 'Gradient Boosting': {'Accuracy': 1.0,
  'Precision': 1.0,
  'Recall': 1.0,
  'F1 Score': 1.0,
  'Confusion Matrix': array([[10,  0],
         [ 0, 11]])},
 'Support Vector Classifier': {'Accuracy': 0.7619047619047619,
  'Precision': 0.75,
  'Recall': 0.8181818181818182,
  'F1 Score': 0.7826086956521738,
  'Confusion Matrix': array([[7, 3],
         [2, 9]])},
 'K-Nearest Neighbors': {'Accuracy': 0.7619047619047619,
  'Precision': 0.6875,
  'Recall': 1.0,
  'F1 Score': 0.8148148148148148,
  'Confusion Matrix': array([[ 5,  5],
         [ 0, 11]])}}

### Spliting Test and Training Sets

In [97]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

label_encoder = LabelEncoder()
eeg_data['specific.disorder'] = label_encoder.fit_transform(eeg_data['specific.disorder'])

X = eeg_data.drop(columns=['specific.disorder'])
y = eeg_data['specific.disorder']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

Training set shape: (83, 1140)
Testing set shape: (21, 1140)


Without CV

In [98]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
label_encoder = LabelEncoder()

eeg_data['sex'] = label_encoder.fit_transform(eeg_data_cleaned['sex'].astype(str))
eeg_data['specific.disorder'] = label_encoder.fit_transform(eeg_data['specific.disorder'].astype(str))

# Handle any remaining missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
eeg_data_imputed = pd.DataFrame(imputer.fit_transform(eeg_data), columns=eeg_data.columns)

# Step 2: Encode Categorical Variables
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode 'specific.disorder' column (e.g., PTSD or Healthy Control)
eeg_data_imputed['specific.disorder'] = label_encoder.fit_transform(eeg_data_imputed['specific.disorder'].astype(str))

# Step 3: Split the Dataset into Features and Target
# Features (X) include all columns except 'specific.disorder'
X = eeg_data_imputed.drop(columns=['specific.disorder'])

# Target (y) is the 'specific.disorder' column
y = eeg_data_imputed['specific.disorder']

# Step 4: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train the classifier on the training data
classifier.fit(X_train, y_train)

# Step 6: Make Predictions and Evaluate the Model
# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Display the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

# Display the shapes of the training and testing sets
print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

Model Accuracy: 0.71
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.70      0.70        10
           1       0.73      0.73      0.73        11

    accuracy                           0.71        21
   macro avg       0.71      0.71      0.71        21
weighted avg       0.71      0.71      0.71        21

Training set shape: (83, 1141)
Testing set shape: (21, 1141)


In [99]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report

eeg_data = pd.read_csv('../Datasets/dataph.csv')

# Step 1.1: Handle Missing Values and Remove Date Columns
# Drop the 'Unnamed: 122' column as it appears redundant or uninformative
eeg_data_cleaned = eeg_data.drop(columns=['Unnamed: 122'])

# Drop columns with date values (e.g., 'eeg.date')
eeg_data_cleaned = eeg_data_cleaned.drop(columns=['eeg.date'])

# Encode categorical variables before handling remaining missing values
# Initialize LabelEncoder
label_encoder = LabelEncoder()


eeg_data_cleaned['sex'] = label_encoder.fit_transform(eeg_data_cleaned['sex'].astype(str))
eeg_data_cleaned['specific.disorder'] = label_encoder.fit_transform(eeg_data_cleaned['specific.disorder'].astype(str))


# Handle any remaining missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
eeg_data_imputed = pd.DataFrame(imputer.fit_transform(eeg_data_cleaned), columns=eeg_data_cleaned.columns)
label_encoder = LabelEncoder()
eeg_data_imputed['sex'] = label_encoder.fit_transform(eeg_data_imputed['sex'].astype(str))
eeg_data_imputed['specific.disorder'] = label_encoder.fit_transform(eeg_data_imputed['specific.disorder'].astype(str))

# Step 3: Split the Dataset into Features and Target
# Features (X) include all columns except 'specific.disorder'
X = eeg_data_imputed.drop(columns=['specific.disorder'])

# Target (y) is the 'specific.disorder' column
y = eeg_data_imputed['specific.disorder']

# Step 4: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train a Basic Classification Model
# Initialize a Random Forest Classifier
classifier = RandomForestClassifier(random_state=42)

# Train the classifier on the training data
classifier.fit(X_train, y_train)

# Step 6: Make Predictions and Evaluate the Model
# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Display the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

# Display the shapes of the training and testing sets
print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

Model Accuracy: 0.90
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.80      0.89        10
           1       0.85      1.00      0.92        11

    accuracy                           0.90        21
   macro avg       0.92      0.90      0.90        21
weighted avg       0.92      0.90      0.90        21

Training set shape: (83, 1145)
Testing set shape: (21, 1145)


In [100]:
eeg_data 

,Unnamed: 0,sex,age,eeg.date,education,IQ,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,AB.A.delta.c.F7,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,F,36.62,2011.1.28,12.0,99.0,Posttraumatic stress disorder,17.603385,17.243334,11.729942,...,94.727123,73.994841,75.159075,72.479599,81.756815,74.578657,78.980627,60.508215,65.113378,69.238767
1,2,F,55.21,2012.4.9,16.0,120.0,Posttraumatic stress disorder,21.714048,19.579805,18.522271,...,83.821476,61.617403,63.964423,73.096645,74.161249,50.432291,77.440228,35.794362,70.592405,48.283439
2,3,F,26.93,2011.3.24,16.0,116.0,Posttraumatic stress disorder,13.371076,14.028142,16.901291,...,70.585726,63.864062,46.127953,65.675414,62.455949,33.114945,56.163998,37.477109,72.732968,46.665464
3,4,M,26.71,2012.7.3,17.0,137.0,Posttraumatic stress disorder,30.473244,13.954586,23.056715,...,87.768539,71.799769,65.845016,67.734909,78.328065,58.141474,68.113967,53.391012,71.111448,67.598506
4,5,F,23.45,2014.6.2,12.0,89.0,Posttraumatic stress disorder,18.488575,19.603144,9.899157,...,70.188502,27.698816,24.181746,34.445899,47.797466,20.737129,47.210870,11.104500,50.442443,36.952529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,100,F,29.04,2018.8.9,16.0,114.0,Healthy control,22.737005,23.110192,20.655895,...,88.188438,48.337650,77.414168,76.804444,51.999675,76.470883,80.365068,48.841509,56.730760,82.017645
100,101,F,25.65,2018.7.2,18.0,118.0,Healthy control,20.081892,17.928614,15.977752,...,99.447826,99.424714,44.181841,61.683927,99.513428,43.007308,62.525177,42.766646,62.843978,39.895496
101,102,F,29.86,2018.7.19,20.0,113.0,Healthy control,46.306229,47.379694,48.808082,...,90.988675,82.588144,91.869140,88.146513,82.027254,87.799612,85.360432,84.867957,90.909785,90.730560
102,103,M,27.00,2017.4.29,18.0,130.0,Healthy control,22.152399,22.698280,19.281922,...,58.751205,46.407702,55.207337,50.262542,56.166800,37.817569,55.401371,31.634053,67.817508,43.937626


In [101]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Load and Preprocess Data
# Load the EEG data CSV file
eeg_data = pd.read_csv('../Datasets/dataph.csv')

eeg_data_cleaned = eeg_data.drop(columns=['Unnamed: 122'])
eeg_data_cleaned = eeg_data_cleaned.drop(columns=['eeg.date'])

# Encode categorical variables before handling remaining missing values
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode 'sex' column (e.g., Male/Female to numerical values)
eeg_data_cleaned['sex'] = label_encoder.fit_transform(eeg_data_cleaned['sex'].astype(str))

# Encode 'specific.disorder' column (e.g., PTSD or Healthy Control)
eeg_data_cleaned['specific.disorder'] = label_encoder.fit_transform(eeg_data_cleaned['specific.disorder'].astype(str))

# Handle any remaining missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
eeg_data_imputed = pd.DataFrame(imputer.fit_transform(eeg_data_cleaned), columns=eeg_data_cleaned.columns)

# Step 3: Split the Dataset into Features and Target
# Features (X) include all columns except 'specific.disorder'
X = eeg_data_imputed.drop(columns=['specific.disorder'])

# Target (y) is the 'specific.disorder' column
y = eeg_data_imputed['specific.disorder']

# Step 4: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train and Evaluate Multiple Classification Models
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Support Vector Classifier': SVC(probability=True, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} Accuracy: {accuracy:.2f}")
    
    # Display the classification report
    report = classification_report(y_test, y_pred)
    print(f"{model_name} Classification Report:\n", report)

# Display the shapes of the training and testing sets
print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

Random Forest Accuracy: 0.90
Random Forest Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.80      0.89        10
         1.0       0.85      1.00      0.92        11

    accuracy                           0.90        21
   macro avg       0.92      0.90      0.90        21
weighted avg       0.92      0.90      0.90        21

Gradient Boosting Accuracy: 1.00
Gradient Boosting Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        10
         1.0       1.00      1.00      1.00        11

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21

Support Vector Classifier Accuracy: 0.76
Support Vector Classifier Classification Report:
               precision    recall  f1-score   support

         0.0       0.78      0.70      0.74        10
    

With CV

In [122]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


eeg_data = pd.read_csv('../Datasets/dataph.csv')
eeg_data_cleaned = eeg_data.drop(columns=['Unnamed: 122'])
eeg_data_cleaned = eeg_data_cleaned.drop(columns=['eeg.date'])

# Encode categorical variables before handling remaining missing values
label_encoder = LabelEncoder()
eeg_data_cleaned['sex'] = label_encoder.fit_transform(eeg_data_cleaned['sex'].astype(str))
eeg_data_cleaned['specific.disorder'] = label_encoder.fit_transform(eeg_data_cleaned['specific.disorder'].astype(str))

# Handle any remaining missing values using SimpleImputer
imputer = SimpleImputer(strategy='mean')
eeg_data_imputed = pd.DataFrame(imputer.fit_transform(eeg_data_cleaned), columns=eeg_data_cleaned.columns)


X = eeg_data_imputed.drop(columns=['specific.disorder'])
y = eeg_data_imputed['specific.disorder']

models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Support Vector Classifier': SVC(probability=True, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

model_results = []

for model_name, model in models.items():
    # Cross-validation
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    
    mean_accuracy = np.mean(cv_scores)
    std_accuracy = np.std(cv_scores)
    
    # Store the results for comparison
    model_results.append({
        'Model': model_name,
        'Mean Accuracy': mean_accuracy,
        'Std Accuracy': std_accuracy
    })

results_df = pd.DataFrame(model_results)
print("\nModel Comparison using Cross-Validation:\n")
print(results_df)

results_df = pd.DataFrame(model_results)
results_df = results_df.sort_values(by='Mean Accuracy', ascending=False)

print("\nModel Comparison using Cross-Validation (sorted by Mean Accuracy):\n")
print(results_df)
best_model = results_df.iloc[0]
print(f"\nThe best performing model is {best_model['Model']} with a Mean Accuracy of {best_model['Mean Accuracy']:.2f}")



Model Comparison using Cross-Validation:

                       Model  Mean Accuracy  Std Accuracy
0              Random Forest       0.750476      0.117325
1          Gradient Boosting       0.902381      0.119618
2  Support Vector Classifier       0.692381      0.114745
3        K-Nearest Neighbors       0.663810      0.083766

Model Comparison using Cross-Validation (sorted by Mean Accuracy):

                       Model  Mean Accuracy  Std Accuracy
1          Gradient Boosting       0.902381      0.119618
0              Random Forest       0.750476      0.117325
2  Support Vector Classifier       0.692381      0.114745
3        K-Nearest Neighbors       0.663810      0.083766

The best performing model is Gradient Boosting with a Mean Accuracy of 0.90


In [119]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

F = 23

# Create the pipeline with SelectKBest and the model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('k_best', SelectKBest(score_func=f_classif, k=F)),
    ('model', RandomForestClassifier(random_state=42))
])

# Use StratifiedKFold for cross-validation to maintain class balance
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation with the pipeline to prevent data leakage
cv_scores = cross_val_score(pipeline, X, y, cv=stratified_kfold, scoring='accuracy')

# Calculate mean and standard deviation of cross-validation accuracy
mean_accuracy = np.mean(cv_scores)
std_accuracy = np.std(cv_scores)

print(f"\nCross-Validation Mean Accuracy: {mean_accuracy:.4f}, Std Accuracy: {std_accuracy:.4f}")

# Train the model on the entire training set using the pipeline
print("\nFitting the pipeline to the entire training set...")
pipeline.fit(X_train, y_train)

# Evaluate the model on the test set
print("\nEvaluating the model on the test set...")
y_pred = pipeline.predict(X_test)
print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))



Cross-Validation Mean Accuracy: 0.9905, Std Accuracy: 0.0190

Fitting the pipeline to the entire training set...

Evaluating the model on the test set...

Test Accuracy: 0.9523809523809523
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.90      0.95        10
         1.0       0.92      1.00      0.96        11

    accuracy                           0.95        21
   macro avg       0.96      0.95      0.95        21
weighted avg       0.96      0.95      0.95        21



In [120]:
# Use SelectKBest to select top k features
k_best = SelectKBest(score_func=f_classif, k=F)  # k=23 as found earlier
X_selected = k_best.fit_transform(X_scaled, y)

# Step 3: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Step 4: Train and Evaluate Multiple Classification Models using Cross-Validation
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Support Vector Classifier': SVC(probability=True, random_state=42)
}

model_results = []

for model_name, model in models.items():
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    # Calculate mean and standard deviation of cross-validation accuracy
    mean_accuracy = np.mean(cv_scores)
    std_accuracy = np.std(cv_scores)
    
    # Store the results for comparison
    model_results.append({
        'Model': model_name,
        'Mean Accuracy': mean_accuracy,
        'Std Accuracy': std_accuracy
    })

# Convert results to DataFrame for easy comparison
results_df = pd.DataFrame(model_results)

# Sort results by Mean Accuracy in descending order
results_df = results_df.sort_values(by='Mean Accuracy', ascending=False)

print("\nModel Comparison using Cross-Validation (sorted by Mean Accuracy):\n")
print(results_df)

# Display the best performing model
best_model_name = results_df.iloc[0]['Model']
print(f"\nThe best performing model is {best_model_name} with a Mean Accuracy of {results_df.iloc[0]['Mean Accuracy']:.2f}")

# Step 5: Hyperparameter Tuning for the Best Model
if best_model_name == 'Random Forest':
    best_model = RandomForestClassifier(random_state=42)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
elif best_model_name == 'Gradient Boosting':
    best_model = GradientBoostingClassifier(random_state=42)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }
elif best_model_name == 'Support Vector Classifier':
    best_model = SVC(probability=True, random_state=42)
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    }

# Perform Grid Search Cross Validation
print(f"\nPerforming hyperparameter tuning for {best_model_name}...")
grid_search = GridSearchCV(best_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Display the best hyperparameters and the best score
print(f"\nBest Hyperparameters for {best_model_name}: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_:.2f}")

# Train the best performing model with the selected features
print("Training the best performing model...")
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Evaluate the model on the test set
print("Evaluating the model on the test set...")
y_pred = best_model.predict(X_test)
print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))



Model Comparison using Cross-Validation (sorted by Mean Accuracy):

                       Model  Mean Accuracy  Std Accuracy
0              Random Forest       0.975000      0.030619
1          Gradient Boosting       0.975000      0.030619
2  Support Vector Classifier       0.927206      0.045469

The best performing model is Random Forest with a Mean Accuracy of 0.97

Performing hyperparameter tuning for Random Forest...
Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time= 

In [116]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

best_k_results = []

# Iterate over a range of k values
for k in range(1, min(X.shape[1], X_train.shape[1]) + 1):
    print(f"Starting cross-validation for k={k}")
    # Create a pipeline to standardize the data, apply feature selection, and train a model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('k_best', SelectKBest(score_func=f_classif, k=k)),
        ('model', RandomForestClassifier(random_state=42))  # You can choose a different model if desired
    ])

    try:
        # Perform cross-validation with StratifiedKFold to maintain class balance
        stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_val_score(pipeline, X_train, y_train, cv=stratified_kfold, scoring='accuracy')
        mean_accuracy = np.mean(cv_scores)
        std_accuracy = np.std(cv_scores)

        print(f"Completed cross-validation for k={k}. Mean Accuracy: {mean_accuracy:.4f}, Std Accuracy: {std_accuracy:.4f}")

        # Store the results for comparison
        best_k_results.append({
            'k': k,
            'Mean Accuracy': mean_accuracy,
            'Std Accuracy': std_accuracy
        })
    except ValueError as e:
        print(f"Error for k={k}: {e}")

# Convert the results to DataFrame for easy comparison
print("\nConverting cross-validation results to DataFrame...")
best_k_results_df = pd.DataFrame(best_k_results)

# Sort by Mean Accuracy and consider Standard Deviation
print("Sorting cross-validation results by Mean Accuracy and Std Accuracy...")
best_k_results_df = best_k_results_df.sort_values(by=['Mean Accuracy', 'Std Accuracy'], ascending=[False, True])

print("\nBest k value comparison using Cross-Validation (sorted by Mean Accuracy and Std Accuracy):\n")
print(best_k_results_df)

# Display the best k value
best_k = best_k_results_df.iloc[0]['k']
print(f"\nThe best value of k is {best_k} with a Mean Accuracy of {best_k_results_df.iloc[0]['Mean Accuracy']:.2f}")

# Use the best k value to transform the data
print(f"Transforming data using the best k={best_k} features...")
k_best_final = SelectKBest(score_func=f_classif, k=int(best_k))
X_selected_final = k_best_final.fit_transform(X_scaled, y)

# Split the data using the selected features
print("Splitting the data into training and testing sets...")
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X_selected_final, y, test_size=0.2, random_state=42)

# Train the best performing model with the selected features
print("Training the best performing model...")
best_model = RandomForestClassifier(random_state=42)
best_model.fit(X_train_final, y_train_final)

# Evaluate the model on the test set
print("Evaluating the model on the test set...")
y_pred = best_model.predict(X_test_final)
print("\nTest Accuracy:", accuracy_score(y_test_final, y_pred))
print("Classification Report:\n", classification_report(y_test_final, y_pred))



Starting cross-validation for k=1
Completed cross-validation for k=1. Mean Accuracy: 0.4353, Std Accuracy: 0.1314
Starting cross-validation for k=2
Completed cross-validation for k=2. Mean Accuracy: 0.4581, Std Accuracy: 0.0608
Starting cross-validation for k=3
Completed cross-validation for k=3. Mean Accuracy: 0.5059, Std Accuracy: 0.1169
Starting cross-validation for k=4
Completed cross-validation for k=4. Mean Accuracy: 0.6029, Std Accuracy: 0.1131
Starting cross-validation for k=5
Completed cross-validation for k=5. Mean Accuracy: 0.6515, Std Accuracy: 0.0752
Starting cross-validation for k=6
Completed cross-validation for k=6. Mean Accuracy: 0.6515, Std Accuracy: 0.0640
Starting cross-validation for k=7
Completed cross-validation for k=7. Mean Accuracy: 0.6529, Std Accuracy: 0.1095
Starting cross-validation for k=8
Completed cross-validation for k=8. Mean Accuracy: 0.6140, Std Accuracy: 0.0816
Starting cross-validation for k=9
Completed cross-validation for k=9. Mean Accuracy: 0.6